In [486]:
import numpy as np
import pandas as pd

In [487]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/ML Driven data challenge /'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/ML Driven data challenge 


In [488]:
Training_set_values=pd.read_csv('Training set values.csv',index_col = "id")
Training_set_values.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [489]:
Training_set_values.shape

(59400, 39)

In [490]:
Training_set_labels=pd.read_csv('Training set labels.csv',index_col = "id")
Training_set_labels.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


In [491]:
Training_set_labels.shape

(59400, 1)

In [492]:
Test_set_values=pd.read_csv('Test set values.csv')
Test_set_values.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [493]:
Test_set_values.shape

(14850, 40)

# Missing values handling

In [494]:
Training_set_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 39 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   date_recorded          59400 non-null  object 
 2   funder                 55765 non-null  object 
 3   gps_height             59400 non-null  int64  
 4   installer              55745 non-null  object 
 5   longitude              59400 non-null  float64
 6   latitude               59400 non-null  float64
 7   wpt_name               59400 non-null  object 
 8   num_private            59400 non-null  int64  
 9   basin                  59400 non-null  object 
 10  subvillage             59029 non-null  object 
 11  region                 59400 non-null  object 
 12  region_code            59400 non-null  int64  
 13  district_code          59400 non-null  int64  
 14  lga                    59400 non-null  object 
 15

In [495]:
Training_set_values.isnull().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

In [496]:
Training_set_values.drop(['funder'], axis=1, inplace=True)
Training_set_values.drop(['subvillage'], axis=1, inplace=True)
Training_set_values.drop(['scheme_name'], axis=1, inplace=True)

In [497]:
Training_set_values.installer.value_counts()

DWE                         17402
Government                   1825
RWE                          1206
Commu                        1060
DANIDA                       1050
                            ...  
TCRS/TWESA                      1
JAPAN                           1
CRISTAN OUTRICH                 1
Manyota primary School          1
HOTEL AND LODGE TANZANIA        1
Name: installer, Length: 2145, dtype: int64

In [498]:
Training_set_values.installer.fillna("DWE",inplace = True)

In [499]:
Training_set_values.public_meeting.value_counts()

True     51011
False     5055
Name: public_meeting, dtype: int64

In [500]:
Training_set_values.public_meeting.fillna(True,inplace = True)

In [501]:
Training_set_values.scheme_management.value_counts()

VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [502]:
Training_set_values.scheme_management.fillna("VWC",inplace = True)

In [503]:
Training_set_values.permit.value_counts()

True     38852
False    17492
Name: permit, dtype: int64

In [504]:
Training_set_values.permit.fillna(True,inplace = True)

In [505]:
Training_set_values.isnull().sum()

amount_tsh               0
date_recorded            0
gps_height               0
installer                0
longitude                0
latitude                 0
wpt_name                 0
num_private              0
basin                    0
region                   0
region_code              0
district_code            0
lga                      0
ward                     0
population               0
public_meeting           0
recorded_by              0
scheme_management        0
permit                   0
construction_year        0
extraction_type          0
extraction_type_group    0
extraction_type_class    0
management               0
management_group         0
payment                  0
payment_type             0
water_quality            0
quality_group            0
quantity                 0
quantity_group           0
source                   0
source_type              0
source_class             0
waterpoint_type          0
waterpoint_type_group    0
dtype: int64

Declare feature vectors

In [506]:
data = Training_set_values.join(Training_set_labels)
data.head()

,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,True,GeoData Consultants Ltd,Other,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,VWC,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [507]:
X = Training_set_values
Y = data["status_group"]

In [508]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=0)

In [509]:
from catboost import Pool, CatBoostClassifier

In [510]:
Training_set_values*1

,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,1,GeoData Consultants Ltd,VWC,0,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,1,GeoData Consultants Ltd,Other,1,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,1,GeoData Consultants Ltd,VWC,1,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,1,GeoData Consultants Ltd,VWC,1,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,1,GeoData Consultants Ltd,VWC,1,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60739,10.0,2013-05-03,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,Kilimanjaro,3,5,Hai,Masama Magharibi,125,1,GeoData Consultants Ltd,Water Board,1,1999,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
27263,4700.0,2011-05-07,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,Iringa,11,4,Njombe,Ikondo,56,1,GeoData Consultants Ltd,VWC,1,1996,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
37057,0.0,2011-04-11,0,DWE,34.017087,-8.750434,Mashine,0,Rufiji,Mbeya,12,7,Mbarali,Chimala,0,1,GeoData Consultants Ltd,VWC,0,0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump


In [511]:
Training_set_values["amount_tsh"] = Training_set_values["amount_tsh"].astype(np.int) 

In [512]:
Training_set_values["longitude"] = Training_set_values["longitude"].astype(np.int) 

In [513]:
Training_set_values["latitude"] = Training_set_values["latitude"].astype(np.int) 

In [514]:
Training_set_values["public_meeting"] = Training_set_values["public_meeting"].astype(np.str) 

In [515]:
Training_set_values["permit"] = Training_set_values["permit"].astype(np.str) 

In [516]:
Training_set_values.dtypes

amount_tsh                int64
date_recorded            object
gps_height                int64
installer                object
longitude                 int64
latitude                  int64
wpt_name                 object
num_private               int64
basin                    object
region                   object
region_code               int64
district_code             int64
lga                      object
ward                     object
population                int64
public_meeting           object
recorded_by              object
scheme_management        object
permit                   object
construction_year         int64
extraction_type          object
extraction_type_group    object
extraction_type_class    object
management               object
management_group         object
payment                  object
payment_type             object
water_quality            object
quality_group            object
quantity                 object
quantity_group           object
source  

In [517]:
Training_set_values.head()

,amount_tsh,date_recorded,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
69572,6000,2011-03-14,1390,Roman,34,-9,none,0,Lake Nyasa,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0,2013-03-06,1399,GRUMETI,34,-2,Zahanati,0,Lake Victoria,Mara,20,2,Serengeti,Natta,280,True,GeoData Consultants Ltd,Other,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25,2013-02-25,686,World vision,37,-3,Kwa Mahundi,0,Pangani,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0,2013-01-28,263,UNICEF,38,-11,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0,2011-07-13,0,Artisan,31,-1,Shuleni,0,Lake Victoria,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,VWC,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [518]:
cat_features = ['date_recorded','gps_height','installer','wpt_name','num_private','basin','region','region_code','district_code','lga','ward','population','public_meeting','recorded_by','scheme_management','permit','construction_year','extraction_type','extraction_type_group','extraction_type_class','management','management_group','payment','payment_type','water_quality','quality_group','quantity','quantity_group','source','source_type','source_class','waterpoint_type','waterpoint_type_group']
text_features = []

In [519]:
train_dataset = Pool(data=X_train,
                     label=y_train,
                     cat_features=cat_features,
                    text_features = text_features)

In [520]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=0)

In [521]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(
    iterations=5, 
    learning_rate=0.1, 
    #loss_function='CrossEntropy'
)


clf.fit(X_train, y_train, 
        cat_features=cat_features, 
        eval_set=(X_val, y_val), 
        verbose=False
)

print('CatBoost model is fitted: ' + str(clf.is_fitted()))
print('CatBoost model parameters:')
print(clf.get_params())

CatBoost model is fitted: True
CatBoost model parameters:
{'iterations': 5, 'learning_rate': 0.1}


In [522]:
from catboost import CatBoostClassifier
clf = CatBoostClassifier(
    iterations=10,
#     verbose=5,
)

clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_val, y_val),
)

Learning rate set to 0.5
0:	learn: 0.8177429	test: 0.8159004	best: 0.8159004 (0)	total: 505ms	remaining: 4.54s
1:	learn: 0.7181330	test: 0.7128174	best: 0.7128174 (1)	total: 991ms	remaining: 3.96s
2:	learn: 0.6765150	test: 0.6691104	best: 0.6691104 (2)	total: 1.42s	remaining: 3.3s
3:	learn: 0.6539440	test: 0.6456444	best: 0.6456444 (3)	total: 1.84s	remaining: 2.77s
4:	learn: 0.6388710	test: 0.6306133	best: 0.6306133 (4)	total: 2.28s	remaining: 2.28s
5:	learn: 0.6289706	test: 0.6202656	best: 0.6202656 (5)	total: 2.7s	remaining: 1.8s
6:	learn: 0.6204590	test: 0.6115014	best: 0.6115014 (6)	total: 3.15s	remaining: 1.35s
7:	learn: 0.6146044	test: 0.6060456	best: 0.6060456 (7)	total: 3.59s	remaining: 898ms
8:	learn: 0.6066974	test: 0.5989537	best: 0.5989537 (8)	total: 4.02s	remaining: 447ms
9:	learn: 0.6027979	test: 0.5957492	best: 0.5957492 (9)	total: 4.45s	remaining: 0us

bestTest = 0.5957492265
bestIteration = 9



In [523]:
print(clf.predict_proba(X_val))

[[0.58945919 0.08902167 0.32151914]
 [0.1265386  0.02848234 0.84497906]
 [0.16147933 0.13599652 0.70252416]
 ...
 [0.42546165 0.09303659 0.48150177]
 [0.22158832 0.05672835 0.72168334]
 [0.43066693 0.12920393 0.44012915]]


In [524]:
print(clf.predict(X_val))

[['functional']
 ['non functional']
 ['non functional']
 ...
 ['non functional']
 ['non functional']
 ['non functional']]


In [525]:
eval_dataset = Pool(data=X_val,
                    label=y_val,
                    cat_features=cat_features)

In [526]:
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=5,
                           loss_function='MultiClass')

In [527]:
model.fit(train_dataset)

0:	learn: 1.0186621	total: 869ms	remaining: 14m 28s
1:	learn: 0.9566143	total: 1.81s	remaining: 15m 4s
2:	learn: 0.9075704	total: 2.68s	remaining: 14m 52s
3:	learn: 0.8666100	total: 3.59s	remaining: 14m 54s
4:	learn: 0.8331004	total: 4.51s	remaining: 14m 56s
5:	learn: 0.8026876	total: 5.33s	remaining: 14m 42s
6:	learn: 0.7779904	total: 6.16s	remaining: 14m 34s
7:	learn: 0.7569500	total: 7.01s	remaining: 14m 29s
8:	learn: 0.7370554	total: 7.78s	remaining: 14m 16s
9:	learn: 0.7175506	total: 8.59s	remaining: 14m 10s
10:	learn: 0.7021363	total: 9.36s	remaining: 14m 1s
11:	learn: 0.6897831	total: 10.2s	remaining: 13m 59s
12:	learn: 0.6796096	total: 11s	remaining: 13m 51s
13:	learn: 0.6688647	total: 11.7s	remaining: 13m 46s
14:	learn: 0.6595666	total: 12.5s	remaining: 13m 40s
15:	learn: 0.6515297	total: 13.4s	remaining: 13m 44s
16:	learn: 0.6434415	total: 14.1s	remaining: 13m 32s
17:	learn: 0.6372923	total: 14.8s	remaining: 13m 28s
18:	learn: 0.6318195	total: 15.7s	remaining: 13m 29s
19:	lea

In [528]:
preds_class = model.predict(eval_dataset)

In [529]:
preds_class

array([['functional'],
       ['non functional'],
       ['functional'],
       ...,
       ['non functional'],
       ['non functional'],
       ['functional']], dtype=object)

In [530]:
preds_proba = model.predict_proba(eval_dataset)

In [531]:
preds_proba

array([[0.46670196, 0.07723294, 0.4560651 ],
       [0.05065275, 0.0162326 , 0.93311465],
       [0.55095735, 0.24080705, 0.2082356 ],
       ...,
       [0.09752229, 0.07568295, 0.82679475],
       [0.03017706, 0.01093936, 0.95888358],
       [0.49880359, 0.16107933, 0.34011707]])

In [532]:
preds_raw = model.predict(eval_dataset, 
                          prediction_type='RawFormulaVal')

In [533]:
preds_raw

array([[ 0.6073067 , -1.1915581 ,  0.5842514 ],
       [-0.59185416, -1.72982617,  2.32168034],
       [ 0.60021624, -0.2274452 , -0.37277104],
       ...,
       [-0.62798246, -0.88151055,  1.50949301],
       [-0.81465761, -1.82937237,  2.64402998],
       [ 0.50441265, -0.62590278,  0.12149012]])

In [534]:
test = pd.read_csv("/content/drive/My Drive/ML Driven data challenge /Test set values.csv", 
                               index_col="id")

In [535]:
test.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [536]:
test.drop(['funder','subvillage','scheme_name'], axis=1, inplace=True)

In [537]:
test.shape

(14850, 36)

In [538]:
test.dtypes

amount_tsh               float64
date_recorded             object
gps_height                 int64
installer                 object
longitude                float64
latitude                 float64
wpt_name                  object
num_private                int64
basin                     object
region                    object
region_code                int64
district_code              int64
lga                       object
ward                      object
population                 int64
public_meeting            object
recorded_by               object
scheme_management         object
permit                    object
construction_year          int64
extraction_type           object
extraction_type_group     object
extraction_type_class     object
management                object
management_group          object
payment                   object
payment_type              object
water_quality             object
quality_group             object
quantity                  object
quantity_g

In [539]:
test.isnull().sum()

amount_tsh                 0
date_recorded              0
gps_height                 0
installer                877
longitude                  0
latitude                   0
wpt_name                   0
num_private                0
basin                      0
region                     0
region_code                0
district_code              0
lga                        0
ward                       0
population                 0
public_meeting           821
recorded_by                0
scheme_management        969
permit                   737
construction_year          0
extraction_type            0
extraction_type_group      0
extraction_type_class      0
management                 0
management_group           0
payment                    0
payment_type               0
water_quality              0
quality_group              0
quantity                   0
quantity_group             0
source                     0
source_type                0
source_class               0
waterpoint_typ

In [540]:
test.installer.fillna("DWE",inplace = True)

In [541]:
test.public_meeting.fillna(True,inplace = True)

In [542]:
test.scheme_management.fillna("VWC",inplace = True)

In [543]:
test.permit.fillna(True,inplace = True)

In [544]:
test_set = Pool(data=test,
                cat_features=cat_features)

In [545]:
res = model.predict(test_set)

In [546]:
len(res)

14850

In [547]:
res[0]

array(['non functional'], dtype=object)

In [548]:
submission_df = pd.read_csv("/content/drive/My Drive/ML Driven data challenge /SubmissionFormat.csv", 
                            index_col="id")

In [549]:
submission_df.head()

,status_group
id,
50785,predicted label
51630,predicted label
17168,predicted label
45559,predicted label
49871,predicted label


In [550]:
submission_df["status_group"] = res

In [552]:
submission_df.to_csv('Result.csv', index=True)